In [44]:
class Field:
    def __init__(self, ftype):
        self.ftype = ftype

    def __set_name__(self, owner, name):
        self.name = name

    # note if a field is invoked from User class, e.g., User.name
    # instance argument here will be None
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance._data.get(self.name)

    def __set__(self, instance, value):
        if not isinstance(value, self.ftype):
            raise TypeError(
                f"{self.name} must be {self.ftype.__name__}"
            )
        instance._data[self.name] = value
        instance._dirty.add(self.name)

    # query operators
    def __eq__(self, value):
        return CmpExpr(self.name, "==", value)

    def __ge__(self, value):
        return CmpExpr(self.name, ">=", value)

    def __le__(self, value):
        return CmpExpr(self.name, "<=", value)

    ########### fill in the code to support > and < (10 points) ############

    def __gt__(self, value):
        return CmpExpr(self.name, ">", value)
    
    def __lt__(self, value):
        return CmpExpr(self.name, "<", value)


In [45]:
class Expr:
    def __bool__(self):
        raise TypeError("Expression cannot be used as boolean")

class CmpExpr(Expr):
    def __init__(self, field, op, value):
        self.field = field
        self.op = op
        self.value = value


In [46]:
import requests
import json
from urllib.parse import urlencode

### replace the following URL with your DB url
### note it does not end with /, that is, no slash after .com
FIREBASE_URL = "https://dsci551-hw1-7868f-default-rtdb.firebaseio.com"

class Document:
    endpoint = None # to be supplied by subclass

    def __init__(self, id, **kwargs):
        self.id = id # id is required
        self._data = {} # attribute-values stored locally
        self._dirty = set() # attributes to be updated on the DB server

        for k, v in kwargs.items():
            setattr(self, k, v) # self.k = v
            # note this will trigger descriptor protocol for field attribute
            # calling its __set__ method

    def save(self):
        """Persist only modified fields."""
        """That is, all fields marked as dirty."""
        """Remember to clear dirty flags after updating the data with server."""
        if not self._dirty:
            return

        ########### fill in the code (30 points) ############
        update_data = {field: self._data[field] for field in self._dirty}
        url = f"{FIREBASE_URL}/{self.endpoint}/{self.id}.json"
        response = requests.patch(url, json=update_data)
        response.raise_for_status()
        self._dirty.clear()


    @classmethod
    def fetch(cls, id):
        """Fetch a document with the given id from server."""
        """Return an object of this class if found."""
        """Return None if not found."""

        ########### fill in the code (25 points) ############
        url = f"{FIREBASE_URL}/{cls.endpoint}/{id}.json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data is None:   
            return None
        instance = cls(id, **data)
        instance._dirty.clear() 
        return instance
            




    @classmethod
    def query(cls, expr):
        """Return a list of documents matching the query."""
        """It does not support orderBy="$key" and orderBy="$value"."""
        """It does not support limit."""

        if not isinstance(expr, CmpExpr):
            raise TypeError("Only one comparison supported")

        base_url = f"{FIREBASE_URL}/{cls.endpoint}.json"

        params = {"orderBy": json.dumps(expr.field)}

        if expr.op == "==":
            params["equalTo"] = json.dumps(expr.value)
        elif expr.op == ">=":
            params["startAt"] = json.dumps(expr.value)
        elif expr.op == "<=":
            params["endAt"] = json.dumps(expr.value)
        elif expr.op == ">":
            params["startAt"] = json.dumps(expr.value + 1)
        elif expr.op == "<":
            params["endAt"] = json.dumps(expr.value - 1)

        ########### fill in the code to support > and < (10 points) ############

        else:
            raise NotImplementedError(expr.op)

        url = f"{base_url}?{urlencode(params)}"

        ########### fill in the code (25 points) ############
        # retrieve from server and construct list of document objects
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data is None:
            return []
        result = []
        for doc_id, doc_data in data.items():
            instance = cls(doc_id, **doc_data)
            instance._dirty.clear()
            if expr.op == ">":
                if instance._data.get(expr.field) == expr.value:
                    continue
            elif expr.op == "<":
                if instance._data.get(expr.field) == expr.value:
                    continue
            result.append(instance)
        return result

    


class User(Document):
    endpoint = "users"

    name = Field(str)
    age = Field(int)
    gender = Field(str)

    def __repr__(self):
        return f"User(id={self.id}, data={self._data})"


In [47]:
u = User(1)
u.gender = "Male"
print(u) # note this only has gender info of User 1

# update User 1 info (kept locally) with the server
u.save()

User(id=1, data={'gender': 'Male'})


In [48]:
u = User.fetch(1) # fetch current value of User 1 from server
print(u)
# modify them locally
u.name = "Bob"
u.age = 30
u.save()  # push updates to the server
print(u)

User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})
User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})


In [49]:
u = User(2, age=20) # specify attribute-value in constructor
print(u)
u.save()
User.fetch(2)

User(id=2, data={'age': 20})


User(id=2, data={'age': 20})

In [50]:
u.age += 1 # update age locally
u.save()
User.fetch(2)

User(id=2, data={'age': 21})

In [51]:
users = User.query(User.age > 30)
users

[]

In [52]:
users = User.query(User.name <= "Mary")
users

[User(id=2, data={'age': 21}),
 User(id=1, data={'age': 30, 'gender': 'Male', 'name': 'Bob'})]